In [2]:
import pandas as pd
import re
from datetime import datetime

In [3]:
PATH = "/Users/alexandrarottenkolber/Documents/02_Hertie_School/Master thesis/Master_Thesis_Hertie/data_analysis/01_data/Wikipedia/output/"

In [4]:
data = pd.read_csv(PATH + "history_revisions_df_complete.csv")

In [61]:
#data.head()

In [66]:
MP_data= pd.read_csv(PATH + "MP_data.csv")

In [67]:
def regex_for_special_chars(string): 
    string = re.sub('%C3%A4', 'ä', string)
    string = re.sub('%C3%B6', 'ö', string)
    string = re.sub('%C3%BC', 'ü', string)
    string = re.sub('%E1%BA%9E', 'ß', string)
    string = re.sub('%C3%9F', 'ß', string)
    string = re.sub('%C3%A8', 'è', string)
    string = re.sub('%C3%96', 'Ö', string)
    string = re.sub('%C3%A9', 'é', string)
    string = re.sub('%C4%B1', 'ı', string)
    string = re.sub('%C4%9F', 'ğ', string)
    string = re.sub('%C5%A1', 'š', string)
    string = re.sub('%C4%87', 'ć', string)
    string = re.sub('%C3%A7', 'ç', string)
    string = re.sub('%C5%BB', 'Ż', string)
    
    
    string = re.split("/", string)[-1]
    string = re.sub('_', ' ', string)
    
    return string

In [68]:
MP_data["wikiurl_clean"] = MP_data["wikiurl"].map(lambda x: regex_for_special_chars(x)) #wikiurl_clean in MP_data and data["title"] match

In [6]:
data = data[["revid","user","timestamp","title"]].copy()

In [7]:
data.head()

,revid,user,timestamp,title
0,221797327,Nuuk,2022-04-05T06:37:32Z,Willy Brandt
1,221792520,Daniel Leon Sch.,2022-04-04T21:39:34Z,Willy Brandt
2,220773568,Redonebird,2022-03-04T11:26:03Z,Willy Brandt
3,220740475,Gloser,2022-03-03T06:52:24Z,Willy Brandt
4,220740267,Julia Heitmann,2022-03-03T06:39:21Z,Willy Brandt


In [8]:
data["year"] = data["timestamp"].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').year)
data["month"] = data["timestamp"].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').month)
data["day"] = data["timestamp"].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').day)
data["date"] = data["timestamp"].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

In [14]:
data.head()

,revid,user,timestamp,title,year,month,day,date
0,221797327,Nuuk,2022-04-05T06:37:32Z,Willy Brandt,2022,4,5,2022-04-05 06:37:32
1,221792520,Daniel Leon Sch.,2022-04-04T21:39:34Z,Willy Brandt,2022,4,4,2022-04-04 21:39:34
2,220773568,Redonebird,2022-03-04T11:26:03Z,Willy Brandt,2022,3,4,2022-03-04 11:26:03
3,220740475,Gloser,2022-03-03T06:52:24Z,Willy Brandt,2022,3,3,2022-03-03 06:52:24
4,220740267,Julia Heitmann,2022-03-03T06:39:21Z,Willy Brandt,2022,3,3,2022-03-03 06:39:21


In [10]:
# revisions cumulativ // revisions year only ?

In [54]:
len(set(data[(data["title"]== "Willy Brandt") & (data["year"]== 2003)]["user"]))

26

In [105]:
counted_per_month = data.groupby(by = ["year", "month", "title"]).nunique().drop(columns = ["day", "date", "timestamp"])

In [106]:
counted_per_month = counted_per_month.reset_index()

In [107]:
counted_per_month[(counted_per_month["title"]== "Willy Brandt") & (counted_per_month["year"]== 2002)]

,year,month,title,revid,user
41,2002,9,Willy Brandt,9,5
82,2002,12,Willy Brandt,3,1


In [108]:
counted_per_month = counted_per_month.rename(columns = {"user": "unique_users", "revid": "no_of_revisions"})

In [109]:
counted_per_month['cum_sum_revisions'] = counted_per_month[["title", "no_of_revisions"]].groupby(by = ["title"]).cumsum()
counted_per_month['cum_sum_users'] = counted_per_month[["title", "unique_users"]].groupby(by = ["title"]).cumsum()

In [110]:
counted_per_month[(counted_per_month["title"]== "Willy Brandt") & (counted_per_month["year"]== 2021)] #2884	1727

,year,month,title,no_of_revisions,unique_users,cum_sum_revisions,cum_sum_users
96203,2021,1,Willy Brandt,14,9,2804,1924
96645,2021,2,Willy Brandt,6,4,2810,1928
97143,2021,3,Willy Brandt,3,3,2813,1931
97928,2021,4,Willy Brandt,20,7,2833,1938
98385,2021,5,Willy Brandt,1,1,2834,1939
98782,2021,6,Willy Brandt,5,5,2839,1944
99297,2021,7,Willy Brandt,2,2,2841,1946
99681,2021,8,Willy Brandt,4,4,2845,1950
100352,2021,9,Willy Brandt,7,7,2852,1957
100883,2021,10,Willy Brandt,2,2,2854,1959


In [111]:
counted_per_year = data.groupby(by = ["year", "title"]).nunique().drop(columns = ["day", "month","date", "timestamp"])

In [112]:
counted_per_year = counted_per_year.reset_index()

In [113]:
counted_per_year = counted_per_year.rename(columns = {"user": "unique_users", "revid": "no_of_revisions"})

In [114]:
counted_per_year[(counted_per_year["title"]== "Willy Brandt") & ((counted_per_year["year"]== 2002) | (counted_per_year["year"]== 2003))]

,year,title,no_of_revisions,unique_users
42,2002,Willy Brandt,12,6
157,2003,Willy Brandt,38,26


In [115]:
len(counted_per_year["title"].unique()), len(counted_per_month["title"].unique())

(2187, 2187)

In [116]:
#wikiurl_clean in MP_data and data["title"] match
keys_df = MP_data[["wikidataid", "wikiurl_clean"]].copy()

In [117]:
counted_per_year

,year,title,no_of_revisions,unique_users
0,2001,Angela Merkel,1,1
1,2001,Franz Müntefering,1,1
2,2001,Fritz Kuhn,1,1
3,2001,Helmut Kohl,2,2
4,2001,Joschka Fischer,2,2
...,...,...,...,...
28402,2022,Xaver Jung,1,1
28403,2022,Yasmin Fahimi,16,9
28404,2022,Yvonne Magwas,5,3
28405,2022,Özcan Mutlu,1,1


In [118]:
counted_per_year['cum_sum_revisions'] = counted_per_year[["title", "no_of_revisions"]].groupby(by = ["title"]).cumsum()
counted_per_year['cum_sum_users'] = counted_per_year[["title", "unique_users"]].groupby(by = ["title"]).cumsum()

In [119]:
counted_per_year[counted_per_year["title"]== "Willy Brandt"] #2867	1972

,year,title,no_of_revisions,unique_users,cum_sum_revisions,cum_sum_users
42,2002,Willy Brandt,12,6,12,6
157,2003,Willy Brandt,38,26,50,32
801,2004,Willy Brandt,234,116,284,148
1892,2005,Willy Brandt,239,142,523,290
3050,2006,Willy Brandt,230,123,753,413
4292,2007,Willy Brandt,229,128,982,541
5652,2008,Willy Brandt,226,150,1208,691
7242,2009,Willy Brandt,171,102,1379,793
8704,2010,Willy Brandt,106,79,1485,872
10461,2011,Willy Brandt,135,95,1620,967


In [132]:
result_counts_per_year = pd.merge(counted_per_year, keys_df, left_on="title", right_on="wikiurl_clean").drop(columns = ["wikiurl_clean"])
result_counts_per_month = pd.merge(counted_per_month, keys_df, left_on="title", right_on="wikiurl_clean").drop(columns = ["wikiurl_clean"])

In [134]:
result_counts_per_month.head(5)

,year,month,title,no_of_revisions,unique_users,cum_sum_revisions,cum_sum_users,wikidataid
0,2001,9,Angela Merkel,1,1,1,1,Q567
1,2002,8,Angela Merkel,2,2,3,3,Q567
2,2002,9,Angela Merkel,1,1,4,4,Q567
3,2002,10,Angela Merkel,3,3,7,7,Q567
4,2003,1,Angela Merkel,1,1,8,8,Q567


In [135]:
#result_counts_per_month.to_csv("/Users/alexandrarottenkolber/Documents/02_Hertie_School/Master thesis/Master_Thesis_Hertie/data_analysis/01_data/Wikipedia/output/history_norevisions_nousers_counted_per_month.csv")

In [136]:
#result_counts_per_year.to_csv("/Users/alexandrarottenkolber/Documents/02_Hertie_School/Master thesis/Master_Thesis_Hertie/data_analysis/01_data/Wikipedia/output/history_norevisions_nousers_counted_per_year.csv")